In [ ]:
# Standard library imports
import math
import os
import sys
import warnings
from typing import List

# Related third-party imports
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.ensemble import (AdaBoostRegressor, GradientBoostingRegressor, 
                              RandomForestRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, 
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller, kpss
from tensorflow.keras.layers import Conv1D, Dense, Flatten, LSTM, SimpleRNN
from tensorflow.keras.models import Sequential

In [ ]:
mlflow.set_tracking_uri("https://dagshub.com/allmamun556/MLOPS_CI_CD_MONITORING.mlflow")
os.environ['MLFLOW_TRACKING_USERNAME'] = 'allmamun556'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'e93226059eb069c73f4c2738187b0a881fb403fa'
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
# os.environ['MLFLOW_TRACKING_USERNAME'] = os.getenv('MLFLOW_TRACKING_USERNAME')
# os.environ['MLFLOW_TRACKING_PASSWORD'] = os.getenv('MLFLOW_TRACKING_PASSWORD')

In [ ]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    Loads data from a CSV file, suppresses warnings, and returns the DataFrame.

    Parameters:
    file_path (str): The path to the CSV file to be loaded.

    Returns:
    pd.DataFrame: The loaded data as a DataFrame.
    """
    
    warnings.filterwarnings('ignore')
    return pd.read_csv(file_path)

In [ ]:
df = load_data('data/Turbine_Data.csv')
df.columns


In [ ]:
df.rename(columns={'Unnamed: 0': 'Time'}, inplace=True)
df

In [ ]:
df.drop(columns=['ControlBoxTemperature', 'WTG'], inplace=True)
missing_values = df.isnull().sum()
missing_values



In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
df = df[df.drop(columns='Time').notna().sum(axis=1) > 0]

In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
correlation_matrix = df.drop(columns='Time').corr()
plt.figure(figsize=(12, 8))

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", 
            square=True, cbar_kws={"shrink": .8}, linewidths=0.5)

plt.title('Correlation Plot')
plt.show()

In [ ]:

df.drop(columns='GeneratorWinding2Temperature', inplace=True)
df.drop(columns='GearboxBearingTemperature', inplace=True)
df.drop(columns='RotorRPM', inplace=True)
df.drop(columns=['Blade2PitchAngle', 'Blade3PitchAngle'], inplace=True)

In [ ]:
print("New DataFrame Shape:", df.shape)
print("Remaining Columns:\n", df.columns)

In [ ]:

correlation_matrix = df.drop(columns=['Time']).corr()
correlation_limit = 0.3 
low_correlation_vars = correlation_matrix[correlation_matrix['ActivePower'].abs() < correlation_limit].index.tolist()
if 'ActivePower' in low_correlation_vars:
    low_correlation_vars.remove('ActivePower')  

df.drop(columns=low_correlation_vars, inplace=True) 

In [ ]:
plt.figure(figsize=(12, 8))
correlation_matrix = df.drop(columns=['Time']).corr()
sns.heatmap(correlation_matrix, annot=True,fmt=".2f")
plt.title('Correlation Plot')
plt.show()

In [ ]:
variables = ['ActivePower', 'BearingShaftTemperature', 'Blade1PitchAngle', 
             'GearboxOilTemperature', 'GeneratorRPM', 'GeneratorWinding1Temperature',
             'HubTemperature', 'ReactivePower', 'WindSpeed']

for var in variables:
    plt.figure(figsize=(12, 5))

    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df[var].dropna(), bins=40, kde=True, color='teal')
    plt.title(f'Histograma de {var}')

    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[var], color='teal')
    plt.title(f'Boxplot de {var}')

    plt.show()

In [ ]:
stats = df[variables].describe().T[['mean', '50%', 'std']]
stats.columns = ['Mean', 'Median', 'Std Dev']

In [ ]:
for var in variables:
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[var] < lower_bound) | (df[var] > upper_bound)]

In [ ]:
df['ActivePower'].fillna(df['ActivePower'].interpolate(method='linear'), inplace=True)


df['BearingShaftTemperature'].fillna(df['BearingShaftTemperature'].median(), inplace=True)

df['Blade1PitchAngle'].fillna(df['Blade1PitchAngle'].median(), inplace=True)


df['GearboxOilTemperature'].fillna(df['GearboxOilTemperature'].median(), inplace=True)

df['GeneratorRPM'].fillna(df['GeneratorRPM'].median(), inplace=True)

df['GeneratorWinding1Temperature'].fillna(df['GeneratorWinding1Temperature'].median(), inplace=True)


df['HubTemperature'].fillna(df['HubTemperature'].median(), inplace=True)


df['ReactivePower'].fillna(df['ReactivePower'].median(), inplace=True)

df['WindSpeed'].fillna(df['WindSpeed'].median(), inplace=True)


missing_values = df.isnull().sum()
missing_values

In [ ]:
# Checking for Stationarity

# Augmented Dickey-Fuller (ADF) Test
result = adfuller(df['ActivePower'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

#KPSS Test
kpss_statistic, p_value, _, critical_values = kpss(df['ActivePower'])
print('KPSS Statistic:', kpss_statistic)
print('p-value:', p_value)
print('Critical Values:', critical_values)



In [ ]:
# Applying differentiation to the 'ActivePower' column

df['active_power_diff'] = df['ActivePower'].diff().dropna()

# Performing the ADF (Augmented Dickey-Fuller) test on the differenced series
# The ADF test checks for stationarity in the time series data.
result_adf = adfuller(df['active_power_diff'].dropna())

# Printing the ADF statistic and p-value for the differenced series

print('ADF Statistic (Differenced):', result_adf[0])
print('p-value (Differenced):', result_adf[1])

# Performing the KPSS (Kwiatkowski-Phillips-Schmidt-Shin) test on the differenced series
# The KPSS test also checks for stationarity but has the null hypothesis that the series is stationary.
kpss_statistic, p_value, _, critical_values = kpss(df['active_power_diff'].dropna())

# Printing the KPSS statistic and p-value for the differenced series

print('KPSS Statistic (Differenced):', kpss_statistic)
print('p-value (Differenced):', p_value)

plt.figure(figsize=(12, 6))
# Plotting the differenced active power data
plt.plot(df['active_power_diff'], label='Differenced Active Power', color='green')
plt.title('Differenced Active Power')
plt.xlabel('Time')
plt.ylabel('Differenced Active Power')
plt.legend()
plt.show()

In [ ]:
# **a) Fitting the ARIMA model**
data= df
df['active_power_diff'].fillna(df['active_power_diff'].mean(), inplace=True)

model_arima = ARIMA(df['active_power_diff'].dropna(), order=(1, 1, 2))
model_arima_fit = model_arima.fit()

# **b) Fitting the Exponential Smoothing model**
model_exp = ExponentialSmoothing(df['active_power_diff'].dropna(), trend='add', seasonal='add', seasonal_periods=12)
model_exp_fit = model_exp.fit()

# **c) Preparing data for other models**
df.to_csv('deepnn.csv')

X = df.drop(columns=['ActivePower', 'Time'])
y = df['ActivePower']

# Removing NaNs
X = X.dropna()
y = y.loc[X.index]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **d) Fitting the Random Forest model**
model_rf = RandomForestRegressor(n_estimators=100)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

# **e) Fitting the Decision Tree model**
model_dt = DecisionTreeRegressor()
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

# **f) Fitting the AdaBoost model**
model_ada = AdaBoostRegressor(n_estimators=100)
model_ada.fit(X_train, y_train)
y_pred_ada = model_ada.predict(X_test)

# **g) Fitting the Gradient Boosting model**
model_gb = GradientBoostingRegressor(n_estimators=100)
model_gb.fit(X_train, y_train)
y_pred_gb = model_gb.predict(X_test)

# **i) Fitting the ANN model**
model_ann = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
model_ann.fit(X_train, y_train)
y_pred_ann = model_ann.predict(X_test)

# **m) Fitting the Polynomial Regression model**
poly = PolynomialFeatures(degree=3)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

model_poly = LinearRegression()
model_poly.fit(X_poly_train, y_train)
y_pred_poly = model_poly.predict(X_poly_test)

# **n) Fitting the Linear Regression model**
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

# **o) Making predictions for ARIMA and Exponential Smoothing**
y_pred_arima = model_arima_fit.forecast(steps=len(y_test))
y_pred_exp = model_exp_fit.forecast(steps=len(y_test))

# **RNN, LSTM, CNN Models - with scaled and sequential data preparation**

# Load your dataset and fill missing values in 'active_power_diff'
#df_cleaned = data.drop(columns=['Unnamed: 0'])
df['active_power_diff'].fillna(df['active_power_diff'].mean(), inplace=True)

# Dropping 'Time' column
df = df.drop(columns=['Time'])

# Define the target and features
X = df.drop(columns=['ActivePower'])
y = df['ActivePower']

# Normalize the data using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Define the sequence length (timesteps)
sequence_length = 10
X_seq, y_seq = [], []

# Creating sequences for model input
for i in range(len(X_scaled) - sequence_length):
    X_seq.append(X_scaled[i:i+sequence_length])
    y_seq.append(y_scaled[i+sequence_length])

X_seq = np.array(X_seq, dtype=np.float64)
y_seq = np.array(y_seq, dtype=np.float64)

# Train-test split for sequential models
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, shuffle=False)

# **r) Fitting the RNN model**
model_rnn = Sequential()
model_rnn.add(SimpleRNN(50, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model_rnn.add(Dense(1))

model_rnn.compile(optimizer='adam', loss='mean_squared_error')
history_rnn = model_rnn.fit(X_train_seq, y_train_seq, epochs=2, batch_size=32, validation_data=(X_test_seq, y_test_seq), verbose=1)

# Predict using RNN
y_pred_rnn = model_rnn.predict(X_test_seq)
y_pred_actual_rnn = scaler_y.inverse_transform(y_pred_rnn)
y_test_actual_rnn = scaler_y.inverse_transform(y_test_seq)

# **s) Fitting the LSTM model**
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model_lstm.add(Dense(1))

model_lstm.compile(optimizer='adam', loss='mean_squared_error')
history_lstm = model_lstm.fit(X_train_seq, y_train_seq, epochs=2, batch_size=32, validation_data=(X_test_seq, y_test_seq), verbose=1)

# Predict using LSTM
y_pred_lstm = model_lstm.predict(X_test_seq)
y_pred_actual_lstm = scaler_y.inverse_transform(y_pred_lstm)
y_test_actual_lstm = scaler_y.inverse_transform(y_test_seq)

# **t) Fitting the CNN model**
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))

model_cnn.compile(optimizer='adam', loss='mean_squared_error')
history_cnn = model_cnn.fit(X_train_seq, y_train_seq, epochs=2, batch_size=32, validation_data=(X_test_seq, y_test_seq), verbose=1)

# Predict using CNN
y_pred_cnn = model_cnn.predict(X_test_seq)
y_pred_actual_cnn = scaler_y.inverse_transform(y_pred_cnn)
y_test_actual_cnn = scaler_y.inverse_transform(y_test_seq)

# **p) Evaluation - Calculating RMSE for each model**

# RMSE for ARIMA and Exponential Smoothing
rmse_arima = mean_squared_error(y_test, y_pred_arima, squared=False)
rmse_exp = mean_squared_error(y_test, y_pred_exp, squared=False)

# RMSE for other models
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
rmse_dt = mean_squared_error(y_test, y_pred_dt, squared=False)
rmse_ada = mean_squared_error(y_test, y_pred_ada, squared=False)
rmse_gb = mean_squared_error(y_test, y_pred_gb, squared=False)
rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
rmse_poly = mean_squared_error(y_test, y_pred_poly, squared=False)
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)

# RMSE for RNN, LSTM, CNN
mse_rnn = mean_squared_error(y_test_actual_rnn, y_pred_actual_rnn)
mae_rnn = mean_absolute_error(y_test_actual_rnn, y_pred_actual_rnn)
r2_rnn = r2_score(y_test_actual_rnn, y_pred_actual_rnn)

mse_lstm = mean_squared_error(y_test_actual_lstm, y_pred_actual_lstm)
mae_lstm = mean_absolute_error(y_test_actual_lstm, y_pred_actual_lstm)
r2_lstm = r2_score(y_test_actual_lstm, y_pred_actual_lstm)

mse_cnn = mean_squared_error(y_test_actual_cnn, y_pred_actual_cnn)
mae_cnn = mean_absolute_error(y_test_actual_cnn, y_pred_actual_cnn)
r2_cnn = r2_score(y_test_actual_cnn, y_pred_actual_cnn)

## Model Evaluation <a name="model-evaluation"></a>
After building our models, we will evaluate their performance using various metrics. We will compare the predictions against the actual outcomes to determine how well our models are performing.

In [ ]:
# Function to calculate and log error metrics and parameters
def evaluate_model_with_mlflow(y_true, y_pred, model_name, params):
    mae = mean_absolute_error(y_true, y_pred)  # Calculate Mean Absolute Error
    mse = mean_squared_error(y_true, y_pred)   # Calculate Mean Squared Error
    rmse = mean_squared_error(y_true, y_pred, squared=False)  # Calculate Root Mean Squared Error
    r2 = r2_score(y_true, y_pred)              # Calculate R² Score

    print(f"{model_name} Performance:")        # Print the model name
    print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}\n")  # Print the metrics

     # Write the metrics to a text file
    with open("metrics.txt", "a") as f:
        f.write(f"{model_name} Performance:\n")
        f.write(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}\n")
        f.write("-" * 30 + "\n")

    # Start MLflow run
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_params(params)
        # Log metrics
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2", r2)
        if model_name=="Random Forest":
            mlflow.sklearn.log_model(model_name, "random_forest_model")

# Example of logging for each model:

# 1. Logging for ARIMA model
arima_params = {'p': 3, 'd': 1, 'q': 0}  # Replace these with your actual ARIMA params
evaluate_model_with_mlflow(y_test, y_pred_arima, "ARIMA", arima_params)

# 2. Logging for Exponential Smoothing model
exp_params = {'trend': 'add', 'seasonal': 'add', 'seasonal_periods': 12}  # Replace with your actual params
evaluate_model_with_mlflow(y_test, y_pred_exp, "Exponential Smoothing", exp_params)

# 3. Logging for Random Forest model
rf_params = {'n_estimators': 100, 'max_depth': 10}  # Replace with your actual RF params
evaluate_model_with_mlflow(y_test, y_pred_rf, "Random Forest", rf_params)

# 1. Logging for Decision Tree model
dt_params = {}  # No specific hyperparameters provided in the example
evaluate_model_with_mlflow(y_test, y_pred_dt, "Decision Tree", dt_params)

# 2. Logging for AdaBoost model
ada_params = {'n_estimators': 100}
evaluate_model_with_mlflow(y_test, y_pred_ada, "AdaBoost", ada_params)

# 3. Logging for Gradient Boosting model
gb_params = {'n_estimators': 100}
evaluate_model_with_mlflow(y_test, y_pred_gb, "Gradient Boosting", gb_params)

# 9. Logging for Polynomial Regression model
poly_params = {'degree': 3}
evaluate_model_with_mlflow(y_test, y_pred_poly, "Polynomial Regression", poly_params)

# 10. Logging for Linear Regression model
lr_params = {}  # No specific hyperparameters for Linear Regression
evaluate_model_with_mlflow(y_test, y_pred_lr, "Linear Regression", lr_params)

# 4. Logging for CNN model
cnn_params = {'filters': 64, 'kernel_size': 2}  # Replace these with the actual CNN parameters
evaluate_model_with_mlflow(y_test_actual_cnn, y_pred_actual_cnn, "CNN", cnn_params)

# 5. Logging for ANN model
ann_params = {'hidden_layer_sizes': (100, 50), 'max_iter': 500}  # Replace these with the actual ANN parameters
evaluate_model_with_mlflow(y_test, y_pred_ann, "ANN", ann_params)

# 6. Logging for RNN model
rnn_params = {'layers': 50, 'activation': 'relu'}  # Replace these with the actual RNN parameters
evaluate_model_with_mlflow(y_test_actual_rnn, y_pred_actual_rnn, "RNN", rnn_params)

# 7. Logging for LSTM model
lstm_params = {'layers': 50, 'activation': 'relu'}  # Replace these with the actual LSTM parameters
evaluate_model_with_mlflow(y_test_actual_lstm, y_pred_actual_lstm, "LSTM", lstm_params)


